In [119]:
import requests 
import re as regex
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import aries_cloudagent.wallet.util as util
import base64
import base58
import uuid

agent1 = {}
agent2 = {}
plugin = {}

## Enter Invite Url:

In [120]:
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiZTgyYzVmYzQtZTVjMC00NzZiLTlhNWQtMzJhZDFiN2UxNjFmIiwgImxhYmVsIjogIk1haW4gKGFkbWluKSIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50MS5sb2NhbGhvc3QiLCAicmVjaXBpZW50S2V5cyI6IFsiOUFWc0FHbVBIQmVSenV2TVNpdFVtOVRnS3ZoaFBoTlNSYk5aRHk2cWppc1IiXX0=


In [121]:
# Process invite url, delete white spaces
agent1['url'] = agent1['url'].replace(" ", "")
# Regex(substitution) to extract only the invite string from url
agent1['invite_string_b64'] = regex.sub(
           r".*(c\_i\=)", 
           r"", 
           agent1['url'])
# Decoding invite string using base64 decoder
agent1['invite_string'] = base64.b64decode(agent1['invite_string_b64'])
# Converting our invite json string into a dictionary 
agent1['invite'] = json.loads(agent1['invite_string'])

print("Decoded invitation:\n")
pp.pprint(agent1['invite'])

Decoded invitation:

{'@id': 'e82c5fc4-e5c0-476b-9a5d-32ad1b7e161f',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Main (admin)',
 'recipientKeys': ['9AVsAGmPHBeRzuvMSitUm9TgKvhhPhNSRbNZDy6qjisR'],
 'serviceEndpoint': 'http://agent1.localhost'}


In [122]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

# print(base58.b58encode(did))
# print(plugin['did'])
print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 3Kz9RqzbMKmnw8252h3um3abrfd7sYRVeW35LbsXNj1xGKC5z4Boz1ywiQ3FvWa28hYXtPvgEJQmTQEUb5ejQbaF

Public key: 
 HYLHdC8r4cTx1ymwPGWNTvggWMq9eriEE7vutn9eqPuy


**Sending a connection request to acapy**

In [123]:
uniqueId = str(uuid.uuid4())
# print(uniqueId)

# our request body
message = {
        "@id":  uniqueId,
        "~transport": {
          "return_route": "all"
        },
        "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/request",
        "label": "Plugin",
        "connection": {
          "DID": plugin['did'],
          "DIDDoc": {
            "@context": "https://w3id.org/did/v1",
            "id": plugin['did'],
            "publicKey": [{
              "id": plugin['did'] + "#keys-1",
              "type": "Ed25519VerificationKey2018",
              "controller": plugin['did'],
              "publicKeyBase58": plugin['public_key_b58']
            }],
            "service": [{
              "id": plugin['did'] + ";indy",
              "type": "IndyAgent",
              "recipientKeys": plugin['public_key_b58'],
              "serviceEndpoint": ""
            }]
          }
        }
      }

**Encoding/packing the request**

In [124]:
# encoding it with aries crypto function using the key that was
# given to us by aca-py in recipientKeys
decodedAcapyKey = base58.b58decode(agent1['invite']['recipientKeys'][0])
ourPrivateKey = plugin['keypair'][1]
# print(plugin['did'])
encodedMessage = \
    crypto.encode_pack_message(json.dumps(message), [decodedAcapyKey], ourPrivateKey)

encodedMessage = encodedMessage.decode("ascii")
# print("Encoded message: \n")
# pp.pprint(encodedMessage)

In [125]:
connectionRequestResponse = requests.post(agent1['invite']['serviceEndpoint'], data=encodedMessage)
assert connectionRequestResponse.text != "", "invalid response from acapy"

**Response from aca-py**

In [126]:
# a bit of a hack to simplify message unpacking,
# decode_pack_message needs a callable object for some reason
def unpackMessage(message, privateKey):
    class FindKey:
        def __init__(self, key):
            self.key = key

        def __call__(self, argument):
            return self.key

    find_key = FindKey(privateKey)   
    return crypto.decode_pack_message(message, find_key)
    
connectionRequestResponseUnpacked = \
    unpackMessage(connectionRequestResponse.text, plugin['keypair'][1])

connectionRequestResponseDict = json.loads(connectionRequestResponseUnpacked[0])

print("Decoded acapy response: \n")
pp.pprint(connectionRequestResponseUnpacked)

Decoded acapy response: 

('{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response", '
 '"@id": "47c446d1-18f1-4a3b-95b8-3dab26a6e905", "~thread": {"thid": '
 '"e83aa68b-0531-4298-a495-b3fbfb7b4ee5"}, "connection~sig": {"@type": '
 '"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single", '
 '"signature": '
 '"3geJZS66rFVz-EaGJIqBaqgi4si7vCHP4WX0PdSqOAtGphnKJFYopeg3lzL4dgud_IcK0iZhrfIm0BKb2cN-Dg==", '
 '"sig_data": '
 '"AAAAAF7DluF7IkRJRCI6ICJWVVZBRE1qVGg0WjFQY240U0dXTVNwIiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjpWVVZBRE1qVGg0WjFQY240U0dXTVNwIiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6VlVWQURNalRoNFoxUGNuNFNHV01TcCMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292OlZVVkFETWpUaDRaMVBjbjRTR1dNU3AiLCAicHVibGljS2V5QmFzZTU4IjogIkdYN2lLOG9zRjJnbVpFR0hDbVlFWlZScmJYWkFQMVpITlpwdkJnYjZ5WEUyIn1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMT

**Unpacking connection data embedded in the response**

In [127]:
sig_data_raw = connectionRequestResponseDict['connection~sig']['sig_data']
#  (this is a hack)replacing artifacts that sometimes happen
sig_data_raw = sig_data_raw.replace("-", "1")
sig_data_raw = sig_data_raw.replace("_", "1")

sig_data_raw = base64.b64decode(sig_data_raw)
# avoid first 8 characters as they are a time signature
sig_data_raw = sig_data_raw[8:]
sig_data_raw = sig_data_raw.decode('ascii')

sig_data = json.loads(sig_data_raw)
pp.pprint(sig_data)

{'DID': 'VUVADMjTh4Z1Pcn4SGWMSp',
 'DIDDoc': {'@context': 'https://w3id.org/did/v1',
            'authentication': [{'publicKey': 'did:sov:VUVADMjTh4Z1Pcn4SGWMSp#1',
                                'type': 'Ed25519SignatureAuthentication2018'}],
            'id': 'did:sov:VUVADMjTh4Z1Pcn4SGWMSp',
            'publicKey': [{'controller': 'did:sov:VUVADMjTh4Z1Pcn4SGWMSp',
                           'id': 'did:sov:VUVADMjTh4Z1Pcn4SGWMSp#1',
                           'publicKeyBase58': 'GX7iK8osF2gmZEGHCmYEZVRrbXZAP1ZHNZpvBgb6yXE2',
                           'type': 'Ed25519VerificationKey2018'}],
            'service': [{'id': 'did:sov:VUVADMjTh4Z1Pcn4SGWMSp;indy',
                         'priority': 0,
                         'recipientKeys': ['GX7iK8osF2gmZEGHCmYEZVRrbXZAP1ZHNZpvBgb6yXE2'],
                         'serviceEndpoint': 'http://agent1.localhost',
                         'type': 'IndyAgent'}]}}


**Adding the connection to connection list**

In [128]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": sig_data['DIDDoc'],
        "myKey": plugin['keypair'][1]
    }
]

def packMessage(message, connection):
    # pass in our private key and recipient key to the encode_pack_message
    decodedRecipientKey = base58.b58decode(connection['DIDDoc']['service'][0]['recipientKeys'][0])
    packedMessage = \
    crypto.encode_pack_message(json.dumps(message), \
                               [decodedRecipientKey], \
                               connection['myKey'])
    return packedMessage.decode('ascii')


pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:VUVADMjTh4Z1Pcn4SGWMSp#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:VUVADMjTh4Z1Pcn4SGWMSp',
             'publicKey': [{'controller': 'did:sov:VUVADMjTh4Z1Pcn4SGWMSp',
                            'id': 'did:sov:VUVADMjTh4Z1Pcn4SGWMSp#1',
                            'publicKeyBase58': 'GX7iK8osF2gmZEGHCmYEZVRrbXZAP1ZHNZpvBgb6yXE2',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:VUVADMjTh4Z1Pcn4SGWMSp;indy',
                          'priority': 0,
                          'recipientKeys': ['GX7iK8osF2gmZEGHCmYEZVRrbXZAP1ZHNZpvBgb6yXE2'],
                          'serviceEndpoint': 'http://agent1.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Main (admin)',
  'myKey': b'tg\xfa\x9d\xb8Y\xa5(\xdd\xac\x08\x03\xa9\x84X~p*\xf9?4\x

### FEATURE DISCOVERY

In [129]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['query'] = '*'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'f8e92e3a-b763-4caf-91e4-ac9a2e6cbb58',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-static-connections/0.1'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-payments/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-issuer/0.1'},
               {'pid': 'https://didcomm.org/routing/1.0'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-basicmessage/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/trust_ping/1.0'},
               {'pid': 'https://didcomm.org/connections/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/routing/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0'},
               {'pid': 'https://didcomm.or

## Dids

In [130]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-list-dids"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'ab2eed0b-bd5c-4fa8-af91-38adf8a539b9',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/list-dids',
 'result': [{'did': 'WjVuoNs4Zc1mK6vXPfqnj5',
             'verkey': 'HCuoA2ewtHWQ297KgJG22VrexYmEbZRCi4aqPkE2cEt9'},
            {'did': 'AH7bQLKyijZkniHvRoLAzu',
             'verkey': '64HqNtZaju4Y5Vd4ZzzBZ4apZqe69gc6AtF9zHr5S9dr'},
            {'did': 'TUNT4Z7kKeYquhmeCZjg8F',
             'verkey': 'FRq4sMiB64yErvd4Rs7k4dmKxzBBVkbFh4Rt6QGmXmbZ'},
            {'did': 'WZndzdvhBc6vXp8hFPrCdL',
             'verkey': 'H7co14Vem8cv3BQ4GLgu3yutTHRRikXfUFofspX9vpDd'},
            {'did': 'qREBhbMGwvNYXYVehH82e',
             'verkey': 'TPabtvsKvYvrGPPohaD6PGy3yn4MxmQbng9BWSgDimx'},
            {'did': '8vxj6PkgTnDZ2bBkyHsxga',
             'verkey': '5KhGWAMLFtxvFjB7YhFZGSjPNnEfDSwEMrHhWB3H2PT9'},
            {'did': '5UupaM5RUM4mUebHuUx2sz',
             'verkey': '3SfHndFMGHHDF3qjEG6zzpvYaBdEo9o3DSxUHAgdjZ1m'},
            {'did': 'PAMhLzhfLCv3DDUP7Yx7ps',
        

## Public did

In [131]:
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '654567eb-bed0-49c0-b1e7-0f92735c2b43',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '~thread': {'thid': 'c2b3f458-79f3-4401-b9d0-3da33615bbaf'}}


In [132]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/schema"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['payload'] = 'qweqwe'

# message['comment'] = "memes"
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '67688d92-6fda-45ae-a7ed-dfa07f1c7365',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/schema',
 'payload': 'qweqwe',
 '~thread': {'thid': 'e70ece56-3698-4f57-832d-35ecf255c894'}}


In [133]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/search_schema"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['type'] = 'schema'

# message['comment'] = "memes"
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{"protected":"eyJlbmMiOiJ4Y2hhY2hhMjBwb2x5MTMwNV9pZXRmIiwidHlwIjoiSldNLzEuMCIsImFsZyI6IkF1dGhjcnlwdCIsInJlY2lwaWVudHMiOlt7ImVuY3J5cHRlZF9rZXkiOiJ6blJic2MzQ3dYU2k1SGtEM2pHaXBDak04ZWFub2pNRnBtTXhxYmFIelFlU2J0dXFNYU9QY3JUdFBVVzRaS0thIiwiaGVhZGVyIjp7ImtpZCI6IkhZTEhkQzhyNGNUeDF5bXdQR1dOVHZnZ1dNcTllcmlFRTd2dXRuOWVxUHV5IiwiaXYiOiJWcUdwTXJLRnF4NFZSS0R3b2JjLTdaenE3QmpOQVVfWSIsInNlbmRlciI6InU3QUxsWWhWVjVMV3Ryaks2a3FjOHFWNzVpSkN5WFg1SVBEcEhCOHllRkpCUU5lQU50T29LazhFMmRLWG9aR0wxVmZDUVozREo1RjFURUpBN0xVR2FIY3ViZi1udzZDcFhUZ3BlbWZCNmJQQUdvOUxqeUw0QUtLQUtFND0ifX1dfQ==","iv":"8P-Llb39aMZ_FjhW","ciphertext":"_ZHWOj8MivvfDRvEJ22gL4MgJ0shx61AmATuGVPnB9jjfbXQ70x9M63pE3CA-G99_2vfkmOL4FHN9B5ojgtZYL8XTkPqmb-MYb8x29LHSEUStRIN37204LTvx24Gk0roOdRjFdPesQkHh-SUDZQHYdE6fRWFmn28nfMqjJUDPQQ_F75eDelOl5u5t4QiNMPf","tag":"9PAeCc9yeYM9rxW6c15Mgg=="}
{'@id': '80135f62-8b76-4e5f-9623-c114fcda41f4',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/search_schema',
 'type': 'schema'}
